In [1]:
%load_ext autoreload
%autoreload

import numpy as np
from tensorflow.keras import *
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
import mne
import pandas as pd
from matplotlib import pyplot as plt 
# from ipywidgets import *
from EEGNet import *

%matplotlib qt5


# Download dataset from https://physionet.org/pn4/eegmmidb/

In [2]:
raws_list = []
for i in range(1,110):
    try:
        runs = [1,2,3,4,5,6, 7,8,9,10, 11,12,13,14] # left vs right hand
        raws_list.append(eegbci.load_data(i, runs))
    except Exception as e:
        print(i,' => ' ,e)

# Load dataset

In [ ]:
print(__doc__)

# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
# tmin, tmax = -1., 4.
event_id = dict(left=1, right=2)
subjects = [i for i in range(1,2)]
runs = [4, 8, 12]  # Motor imagery: left hand vs right hand

raw_fnames = []
for i in subjects:
    for f in eegbci.load_data(i, runs):
        raw_fnames.append(f)
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])

# Visualization & Underestanding

In [5]:
raw.info

<Info | 16 non-empty fields
    bads : list | 0 items
    ch_names : list | Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., ...
    chs : list | 64 items (EEG: 64)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 80.0 Hz
    meas_date : tuple | 2009-08-12 16:15:00 GMT
    nchan : int | 64
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 160.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplotter

In [6]:
raw.get_data().shape

(64, 2902240)

In [7]:
# strip channel names of "." characters
raw.rename_channels(lambda x: x.strip('.'))

## Plot the montage of sensors

In [8]:
montage = mne.channels.read_montage('biosemi64')
raw.set_montage(montage)
# raw.plot_sensors('3d')
%matplotlib qt5
# raw.plot_psd(fmax=30)

<ipython-input-8-55f34095d476>:2: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['T9', 'T10']. Their position has been left untouched.
  raw.set_montage(montage)


In [9]:
events_from_annotations(raw, event_id=dict(T1=1, T2=2))[0].shape

Used Annotations descriptions: ['T1', 'T2']


(2205, 3)

In [10]:
# Apply band-pass filter
raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')

events, _ = events_from_annotations(raw, event_id=dict(T1=1, T2=2))

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
                   exclude='bads')

# Read epochs (train will be done only between 1 and 2s)
# Testing will be done with a running classifier
epochs = Epochs(raw, events, event_id, -1, 4, proj=True, picks=picks,
                baseline=None, preload=True)

Filtering raw data in 147 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 265 samples (1.656 sec)

Used Annotations descriptions: ['T1', 'T2']
2205 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2205 events and 801 original time points ...
5 bad epochs dropped


In [11]:
epochs.get_data().shape

(2200, 64, 801)

# Train-set & Labels

In [12]:
epochs_train = epochs.copy().crop(tmin=1., tmax=2)
labels = epochs_train.events[:, -1] - 1

In [13]:
epochs_train.get_data().shape
# labels.shape

(2200, 64, 161)

In [14]:
# df=pd.DataFrame(events, columns = ['time','x','event'])
# df.head()

In [15]:
# fig, ax = plt.subplots()

# ax.scatter(df.index, df['event'])

# indexA = np.array(df.index[df['event']==3])
# for i in indexA:
#     ax.axvline(x=i, ymin=0, ymax=1, c='r', linestyle='--')

In [16]:
# mne.viz.plot_events(events, raw.info['sfreq'], raw.first_samp,
#                     event_id=event_id)

# plt.show()

In [17]:
# df.hist()

In [18]:
# # Extract data from the first 5 channels
# n_ch = 5
# sfreq = raw.info['sfreq']
# data, times = raw[:n_ch, int(sfreq * 1):int(sfreq * 100)]

# fig = plt.subplots(figsize=(10,8))
# plt.plot(times, data.T);
# plt.xlabel('Seconds')
# plt.ylabel('$\mu V$')
# plt.title('Channels: 1-' + str(n_ch));
# plt.legend(raw.ch_names[:n_ch]);

## Train test spilt 
### using mne method

In [19]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()
cv = ShuffleSplit(10, test_size=0.25, random_state=42)
cv_split = cv.split(epochs_data_train)

In [20]:
# c = []
# for train_idx, test_idx in cv_split:
#     y_train, y_test = labels[train_idx], labels[test_idx]
#     print(epochs_data_train[train_idx])
#     c.append(csp.fit_transform(epochs_data_train[train_idx], y_train))
# # c[0]

# Classifier

In [21]:
# classifier = svm.SVC(gamma='auto')
# classifier = LogisticRegression(solver='lbfgs')
# classifier = LinearDiscriminantAnalysis()
# classifier = GaussianNB()
# classifier = LassoLarsCV()

In [22]:
# csp = CSP(n_components=2, reg=None, log=True, norm_trace=False)

# # Use scikit-learn Pipeline with cross_val_score function
# clf = Pipeline([('CSP', csp), ('Classifier', classifier)])
# scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

# # Printing the results
# class_balance = np.mean(labels == labels[0])
# class_balance = max(class_balance, 1. - class_balance)
# print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
#                                                           class_balance))

# # plot CSP patterns estimated on full data for visualization
# csp.fit_transform(epochs_data, labels)

# layout = read_layout('EEG1005')
# csp.plot_patterns(epochs.info, layout=layout, ch_type='eeg',
#                   units='Patterns (AU)', size=2)

In [23]:
labels.shape
labels = labels.reshape((labels.shape[0],1))
epochs_data_train = epochs_data_train.reshape((epochs_data_train.shape[0],1, epochs_data_train.shape[1], epochs_data_train.shape[2]))

In [24]:
X_train, X_test, y_train, y_test = train_test_split(epochs_data_train, labels, test_size=0.2, random_state=42)

print('shape of train-set-X: {} \nshape of test-set-X: {}'.format(X_train.shape, X_test.shape))
print('shape of train-set-y: {} \nshape of test-set-y: {}'.format(y_train.shape, y_test.shape))

shape of train-set-X: (1760, 1, 64, 161) 
shape of test-set-X: (440, 1, 64, 161)
shape of train-set-y: (1760, 1) 
shape of test-set-y: (440, 1)


In [25]:
%autoreload
import EEGNet
batch_size = 50
# model = EEGNet.my_EEGNet(input_shape=(64,161), batch_size=None,n_classes=2)
# model = EEGNet.EEGNet(2,Samples=161, F1=8, kernLength=20)
model = EEGNet.EEGNet(2, Chans = 64, Samples = 161, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
# model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
optimizer = optimizers.Adam(lr=0.0001)
model.compile(optimizer ,'sparse_categorical_crossentropy', metrics=['acc'])

In [27]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 64, 161)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 64, 161)        512       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 8, 64, 161)        32        
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 16, 1, 161)        1024      
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 16, 1, 161)        64        
_________________________________________________________________
activation (Activation)      (None, 16, 1, 161)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 16, 1, 40)         0         
__________

In [29]:
history = model.fit(X_train, y_train, batch_size=10,epochs=100,validation_data=(X_test, y_test))

Train on 1760 samples, validate on 440 samples
Epoch 1/100


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d/Conv2D}}]]
	 [[{{node metrics/acc/div_no_nan}}]]

In [83]:
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])

(2200, 1, 64, 801)